# Cleanup: Remove All Production Resources

This notebook cleans up all resources created in the previous notebooks:
- AgentCore Runtime and ECR repository (from notebook 2)
- VPC infrastructure (from notebook 1)

⚠️ **Warning**: This will permanently delete all resources. Make sure you want to proceed.

In [ ]:
import json
import boto3
import time
from botocore.exceptions import ClientError

# Load configuration files
try:
    with open('vpc_config.json', 'r') as f:
        vpc_config = json.load(f)
    print("✅ VPC configuration loaded")
except FileNotFoundError:
    print("⚠️  VPC configuration not found")
    vpc_config = None

try:
    with open('deployment_info.json', 'r') as f:
        deployment_info = json.load(f)
    print("✅ Deployment information loaded")
except FileNotFoundError:
    print("⚠️  Deployment information not found")
    deployment_info = None

## 1. Cleanup AgentCore Runtime and ECR

In [ ]:
if deployment_info:
    region = deployment_info['region']
    
    # Initialize clients
    agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
    ecr_client = boto3.client('ecr', region_name=region)
    
    print("🧹 Cleaning up AgentCore Runtime...")
    
    # Delete AgentCore Runtime
    try:
        runtime_delete_response = agentcore_control_client.delete_agent_runtime(
            agentRuntimeId=deployment_info['agent_id']
        )
        print(f"✅ AgentCore Runtime deletion initiated: {deployment_info['agent_id']}")
    except ClientError as e:
        print(f"⚠️  Error deleting AgentCore Runtime: {e}")
    
    # Delete ECR repository
    try:
        ecr_repo_name = deployment_info['ecr_uri'].split('/')[-1]
        response = ecr_client.delete_repository(
            repositoryName=ecr_repo_name,
            force=True
        )
        print(f"✅ ECR repository deleted: {ecr_repo_name}")
    except ClientError as e:
        print(f"⚠️  Error deleting ECR repository: {e}")
        
else:
    print("⏭️  No AgentCore deployment to cleanup")

## 2. Cleanup VPC Infrastructure

In [ ]:
if vpc_config:
    region = vpc_config['region']
    ec2 = boto3.client('ec2', region_name=region)
    
    print("🧹 Cleaning up VPC infrastructure...")
    
    # Delete VPC Endpoints
    try:
        endpoints = ec2.describe_vpc_endpoints(
            Filters=[{'Name': 'vpc-id', 'Values': [vpc_config['vpc_id']]}]
        )['VpcEndpoints']
        
        for endpoint in endpoints:
            ec2.delete_vpc_endpoints(VpcEndpointIds=[endpoint['VpcEndpointId']])  # PLURAL and array
            print(f"✅ VPC Endpoint deleted: {endpoint['VpcEndpointId']}")
    except ClientError as e:
        print(f"⚠️  Error deleting VPC endpoints: {e}")
    
    # Wait for endpoints to be deleted
    print("⏳ Waiting for VPC endpoints to be deleted...")
    time.sleep(30)
    
else:
    print("⏭️  No VPC configuration to cleanup")
    vpc_config = {}

## 3. Delete NAT Gateways

In [ ]:
if vpc_config and 'nat_gateways' in vpc_config:
    print("🧹 Deleting NAT Gateways...")
    
    for nat_name, nat_id in vpc_config['nat_gateways'].items():
        try:
            ec2.delete_nat_gateway(NatGatewayId=nat_id)
            print(f"✅ NAT Gateway deletion initiated: {nat_id}")
        except ClientError as e:
            print(f"⚠️  Error deleting NAT Gateway {nat_id}: {e}")
    
    # Wait for NAT Gateways to be deleted
    print("⏳ Waiting for NAT Gateways to be deleted...")
    time.sleep(60)
else:
    print("⏭️  No NAT Gateways to delete")

## 4. Delete Route Tables

In [ ]:
if vpc_config and 'vpc_id' in vpc_config:
    print("🧹 Deleting Route Tables...")
    
    try:
        # Get all route tables for the VPC
        route_tables = ec2.describe_route_tables(
            Filters=[{'Name': 'vpc-id', 'Values': [vpc_config['vpc_id']]}]
        )['RouteTables']
        
        for rt in route_tables:
            # Skip main route table
            if not any(assoc.get('Main', False) for assoc in rt.get('Associations', [])):
                rt_id = rt['RouteTableId']
                
                # First, disassociate from subnets
                for assoc in rt.get('Associations', []):
                    if 'SubnetId' in assoc and not assoc.get('Main', False):
                        try:
                            ec2.disassociate_route_table(
                                AssociationId=assoc['RouteTableAssociationId']
                            )
                            print(f"✅ Route table association removed: {assoc['RouteTableAssociationId']}")
                        except ClientError as e:
                            print(f"⚠️  Error removing association: {e}")
                
                # Then delete the route table
                try:
                    ec2.delete_route_table(RouteTableId=rt_id)
                    print(f"✅ Route Table deleted: {rt_id}")
                except ClientError as e:
                    print(f"⚠️  Error deleting Route Table {rt_id}: {e}")
    except ClientError as e:
        print(f"⚠️  Error listing route tables: {e}")
else:
    print("⏭️  No Route Tables to delete")

## 5. Delete Internet Gateway

In [ ]:
if vpc_config and 'vpc_id' in vpc_config:
    print("🧹 Deleting Internet Gateway...")
    
    try:
        # Get Internet Gateways attached to VPC
        igws = ec2.describe_internet_gateways(
            Filters=[{'Name': 'attachment.vpc-id', 'Values': [vpc_config['vpc_id']]}]
        )['InternetGateways']
        
        for igw in igws:
            igw_id = igw['InternetGatewayId']
            # Detach from VPC
            ec2.detach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_config['vpc_id'])
            # Delete IGW
            ec2.delete_internet_gateway(InternetGatewayId=igw_id)
            print(f"✅ Internet Gateway deleted: {igw_id}")
    except ClientError as e:
        print(f"⚠️  Error deleting Internet Gateway: {e}")
else:
    print("⏭️  No Internet Gateway to delete")

## 6. Delete Subnets

In [ ]:
if vpc_config and 'subnets' in vpc_config:
    print("🧹 Deleting Subnets...")
    
    for subnet_name, subnet_id in vpc_config['subnets'].items():
        try:
            ec2.delete_subnet(SubnetId=subnet_id)
            print(f"✅ Subnet deleted: {subnet_id} ({subnet_name})")
        except ClientError as e:
            print(f"⚠️  Error deleting subnet {subnet_id}: {e}")
else:
    print("⏭️  No Subnets to delete")

## 7. Delete Security Groups

In [ ]:
if vpc_config and 'security_groups' in vpc_config:
    print("🧹 Deleting Security Groups...")
    
    for sg_name, sg_id in vpc_config['security_groups'].items():
        try:
            ec2.delete_security_group(GroupId=sg_id)
            print(f"✅ Security Group deleted: {sg_id} ({sg_name})")
        except ClientError as e:
            print(f"⚠️  Error deleting security group {sg_id}: {e}")
else:
    print("⏭️  No Security Groups to delete")

## 8. Delete VPC

In [ ]:
if vpc_config and 'vpc_id' in vpc_config:
    print("🧹 Deleting VPC...")
    
    try:
        ec2.delete_vpc(VpcId=vpc_config['vpc_id'])
        print(f"✅ VPC deleted: {vpc_config['vpc_id']}")
    except ClientError as e:
        print(f"⚠️  Error deleting VPC: {e}")
else:
    print("⏭️  No VPC to delete")

## 9. Release Elastic IPs

In [ ]:
if vpc_config and 'project_name' in vpc_config:
    print("🧹 Releasing Elastic IPs...")
    
    try:
        # Get all EIPs with project tag
        addresses = ec2.describe_addresses(
            Filters=[{'Name': 'tag:Project', 'Values': [vpc_config['project_name']]}]
        )['Addresses']
        
        for addr in addresses:
            if 'AllocationId' in addr:
                ec2.release_address(AllocationId=addr['AllocationId'])
                print(f"✅ Elastic IP released: {addr['AllocationId']}")
    except ClientError as e:
        print(f"⚠️  Error releasing Elastic IPs: {e}")
else:
    print("⏭️  No Elastic IPs to release")

## 10. Cleanup Configuration Files

In [ ]:
import os

print("🧹 Cleaning up configuration files...")

files_to_remove = [
    'vpc_config.json',
    'deployment_info.json',
    'deployment_config.json',
    'production_agent.py',
    'deploy_commands.sh'
]

for file_name in files_to_remove:
    try:
        if os.path.exists(file_name):
            os.remove(file_name)
            print(f"✅ File removed: {file_name}")
        else:
            print(f"⏭️  File not found: {file_name}")
    except Exception as e:
        print(f"⚠️  Error removing {file_name}: {e}")

## Summary

In [ ]:
print("\n🎉 Cleanup completed!")
print("\nResources cleaned up:")
print("✅ AgentCore Runtime and ECR repository")
print("✅ VPC Endpoints")
print("✅ NAT Gateways and Elastic IPs")
print("✅ Route Tables")
print("✅ Internet Gateway")
print("✅ Subnets")
print("✅ Security Groups")
print("✅ VPC")
print("✅ Configuration files")
print("\n💡 All production resources have been removed.")